In [ ]:
TOKEN = "<YOUR API_KEY>"

# База

In [13]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="just-ai/claude/claude-3-5-sonnet",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"
user_template = "Hello everybody"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

In [15]:
prompt_template.invoke({'language': 'Russian'})

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Russian', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello everybody', additional_kwargs={}, response_metadata={})])

In [16]:
chain = prompt_template | llm
chain.invoke(
    {
        'language': 'Russian'
    }
)

AIMessage(content='Здравствуйте все / Привет всем\n\nNote: "Здравствуйте" is more formal, while "Привет" is informal.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 17, 'total_tokens': 59, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'claude-3-5-sonnet-20241022', 'system_fingerprint': None, 'id': '8d3d438d-8c71-4e79-bfe0-5122ce4a11a3', 'finish_reason': None, 'logprobs': None}, id='run-6ee1df59-ede3-4a62-a3df-2c0769059bea-0', usage_metadata={'input_tokens': 17, 'output_tokens': 42, 'total_tokens': 59, 'input_token_details': {}, 'output_token_details': {}})

# Генерация кода для Export-шага

In [4]:
MODEL = "just-ai/openai-proxy/gpt-4.1"
TEMPERATURE = 0

In [5]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)

In [14]:
from typing import List, Dict, Optional
from pydantic import BaseModel

class DataSource(BaseModel):
    name: str
    description: str
    data_schema: Dict[str, str] # column_name: type
    type: str  # 'table', 'csv', 'api', etc.
    database: str
    access_method: str | None = None
    limitations: Optional[str] | None = None
    recommendations: List[str] | None = None
    connection_params: Dict[str, str] | None = None


ds_customers = DataSource(
        name="customers",
        description="Таблица клиентов",
        data_schema={"customer_id": "Int64", 
                     "name": "Text", 
                     "position": "Text",
                     "Age": "Int64",
                     "tariff": "Json"},
        type="table",
        database="ClickHouse",
    )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = (
    "You are an experienced Python data engineer writing code for an Airflow DAG. " \
)

# user_template = (
#     "Implement a function def export_data_from_source(**context) -> None for an Airflow DAG.\n"
#     "- The data source has the following properties:\n"
#     "  - database: {database}\n"
#     "  - table name: {name}\n"
#     "  - data schema: {data_schema}\n"
#     "- Use the appropriate Airflow connection for the database type (for example, PostgresHook(\"<name>_source\") for PostgreSQL or ClickHouseHook(\"<name>_source)\" for Clickhouse).\n"
#     "- Use only standard and popular open-source Python libraries (such as pandas, psycopg2). \n"
#     "- Save the extracted data to a file in a suitable format (CSV, JSON, Parquet, etc). Preferably CSV.\n"
#     "- Add a docstring in Russian that describes what the function does.\n"
#     "- Do not add any comments or explanations outside the function code.\n"
#     "- Return only the function code."
# )

user_template = (
    "Implement a function def export_and_load_data_from_source(**context) -> None for an Airflow DAG.\n"
    "- The data source has the following properties: {data_source} \n"
    "- Use the appropriate Airflow connection for the database type (for example, PostgresHook(\"<name>_source\") for PostgreSQL or ClickHouseHook(\"<name>_source)\" for Clickhouse).\n"
    "- Use only standard and popular open-source Python libraries (such as pandas, psycopg2). \n"
    "- Save the extracted data to a file in a suitable format (CSV, JSON, Parquet, etc). Preferably CSV.\n"
    "- Add a docstring in Russian that describes what the function does.\n"
    "- Do not add any comments or explanations outside the function code.\n"
    "- Return only the function code."
)


prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

In [16]:
chain = prompt_template | llm
# result = chain.invoke(
#     {"database": ds_customers.database,
#      "name": ds_customers.name,
#      "data_schema": ds_customers.data_schema}
# )

result = chain.invoke(
    {"data_source": ds_customers}
)

In [17]:
result.content

'```python\ndef export_data_from_source(**context) -> None:\n    """\n    Экспортирует данные из источника ClickHouse, используя соответствующий Airflow Hook.\n    Данные из таблицы \'customers\' сохраняются в формате CSV.\n    """\n    from airflow.providers.clickhouse.hooks.clickhouse import ClickHouseHook\n    import pandas as pd\n\n    # Создание подключения к ClickHouse\n    hook = ClickHouseHook(\'clickhouse_default\')\n\n    # SQL запрос для выборки данных\n    sql = "SELECT * FROM customers"\n\n    # Выполнение запроса и загрузка данных в DataFrame\n    df = hook.get_pandas_df(sql)\n\n    # Сохранение данных в CSV файл\n    df.to_csv(\'/path/to/output/customers_data.csv\', index=False)\n```'

In [ ]:
import re

def clean_code(code_str: str) -> str:
    # убрать обрамление ``` или ```python и оставить только содержимое
    pattern = r"```(?:python)?\n(.*?)```"
    matches = re.findall(pattern, code_str, re.DOTALL)
    if matches:
        # если несколько блоков, объединяем их через 2 перевода строки
        return "\n\n".join(match.strip() for match in matches)
    return code_str.strip()

cleaned_result = clean_code(result.content)

In [19]:
cleaned_result

'def export_data_from_source(**context) -> None:\n    """\n    Экспортирует данные из источника ClickHouse, используя соответствующий Airflow Hook.\n    Данные из таблицы \'customers\' сохраняются в формате CSV.\n    """\n    from airflow.providers.clickhouse.hooks.clickhouse import ClickHouseHook\n    import pandas as pd\n\n    # Создание подключения к ClickHouse\n    hook = ClickHouseHook(\'clickhouse_default\')\n\n    # SQL запрос для выборки данных\n    sql = "SELECT * FROM customers"\n\n    # Выполнение запроса и загрузка данных в DataFrame\n    df = hook.get_pandas_df(sql)\n\n    # Сохранение данных в CSV файл\n    df.to_csv(\'/path/to/output/customers_data.csv\', index=False)'

In [20]:
with open(f"results/export/airflow_template_3_{MODEL.split('/')[-1].replace('-', '_')}.py", 
          "w", encoding="utf-8") as f:
    f.write(cleaned_result)

# Собираем цепочку для генерации пайплайна

#### разбираем airflow args

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

MODEL = "just-ai/openai-proxy/gpt-4o"
TEMPERATURE = 0

llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)

In [19]:
from pydantic import BaseModel

class BusinessProcess(BaseModel):
    name: str = ""
    schedule: str = ""

business_process = BusinessProcess(
    name="Аналитика заказов интернет-магазина",
    schedule="Ежедневно по ночам (в промежуток с 01:00 по 03:00), начиная с 1 февраля 2025 года"
)

In [20]:
# Генерируем schedule_interval и start_date на основе business_process
system_template = (
    "You are an experienced data engineer. Your task is to choose the correct Airflow schedule_interval and start_date for a DAG, "
    "based on the following business process and recommendations. \n"
    "Return only the values in Python code format"
    "(for example: schedule_interval=\"@daily\"\nstart_date=datetime(2024, 1, 1))."
)
user_template = (
    "Business process:\n"
    "- name: {name}\n"
    "- schedule: {schedule}\n"
)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", user_template)
])
chain = prompt_template | llm

result_for_args = chain.invoke({
    "name": getattr(business_process, "name", "Анализ"),
    "schedule": getattr(business_process, "schedule", "0 0 * * *"),
})

In [94]:
cleaned_args_code = clean_code(result_for_args.content)
cleaned_args_code

'schedule_interval="0 1 * * *"\nstart_date=datetime(2025, 2, 1)'

In [95]:
cleaned_args_code.splitlines()

['schedule_interval="0 1 * * *"', 'start_date=datetime(2025, 2, 1)']

#### добавляем код для функции moving

In [ ]:
from typing import List, Dict, Optional
from pydantic import BaseModel

class DataSource(BaseModel):
    name: str
    description: str
    data_schema: Dict[str, str] # column_name: type
    type: str  # 'table', 'csv', 'api', etc.
    database: str
    access_method: str | None = None
    limitations: Optional[str] | None = None
    recommendations: List[str] | None = None
    connection_params: Dict[str, str] | None = None


ds_orders = DataSource(
    name="orders",
    description="Таблица заказов",
    data_schema={"order_id": "Int64",
                 "product_id": "Int64",
                 "timestamp": "datetime",
                 "customer_id": "Int64",
                 "money": "numeric"},
    type="table",
    database="PostgreSQL",
    access_method=None,
    recommendations=["использовать фильтрацию по дате (timestamp)"]
)

ds = [ds_customers, ds_orders]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI

MODEL = "just-ai/gemini/gemini-2.5-pro"
# MODEL = "just-ai/openai-proxy/gpt-4o"
TEMPERATURE = 0

llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)

system_template = (
    "You are an experienced Python data engineer writing code for an Airflow DAG. " \
)

user_template = (
    "Implement a function def moving_data_from_source_to_dwh(**context) -> None for an Airflow DAG.\n"
    "- The data source has the following properties:\n"
    "  - database: {database}\n"
    "  - table name: {name}\n"
    "  - data schema: {data_schema}\n"
    "- For analytics database use ClickHouseHook('clickhouse_dwh')"
    "- Use the appropriate Airflow connection for the database type (for example, PostgresHook(\"<name>_source\") for PostgreSQL or ClickHouseHook(\"<name>_source)\" for Clickhouse).\n"
    "- Use only standard and popular open-source Python libraries (such as pandas, psycopg2). \n"
    "- Add a docstring in Russian that describes what the function does.\n"
    "- Import all needed libraries inside function.\n"
    "- Do not add any comments, code or explanations outside the function code.\n"
    "- Return only the function code."
)

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

chain = prompt_template | llm

result_moving = chain.invoke(
    {"database": ds_orders.database,
     "name": ds_orders.name,
     "data_schema": ds_orders.data_schema}
)

In [159]:
cleaned_code_moving = clean_code(result_moving.content)
cleaned_code_moving

'def moving_data_from_source_to_dwh(**context) -> None:\n    """\n    Извлекает данные из таблицы \'orders\' в источнике PostgreSQL\n    и загружает их в таблицу \'orders\' в ClickHouse DWH.\n\n    Предполагается, что структура таблицы \'orders\' в ClickHouse\n    соответствует схеме источника:\n    (order_id Int64, product_id Int64, timestamp DateTime, customer_id Int64, money Decimal).\n    """\n    import pandas as pd\n    from airflow.providers.postgres.hooks.postgres import PostgresHook\n    from airflow.providers.clickhouse.hooks.clickhouse import ClickHouseHook\n\n    # Define connection IDs and table names\n    postgres_conn_id = "postgres_source"  # Replace with your actual Postgres connection ID\n    clickhouse_conn_id = "clickhouse_dwh"\n    source_table = "orders"\n    target_table = "orders" # Assuming the target table has the same name\n\n    # Define source schema explicitly for clarity, though hook handles types\n    source_schema = {\n        \'order_id\': \'Int64\',\n

In [160]:
cleaned_code_moving

'def moving_data_from_source_to_dwh(**context) -> None:\n    """\n    Извлекает данные из таблицы \'orders\' в источнике PostgreSQL\n    и загружает их в таблицу \'orders\' в ClickHouse DWH.\n\n    Предполагается, что структура таблицы \'orders\' в ClickHouse\n    соответствует схеме источника:\n    (order_id Int64, product_id Int64, timestamp DateTime, customer_id Int64, money Decimal).\n    """\n    import pandas as pd\n    from airflow.providers.postgres.hooks.postgres import PostgresHook\n    from airflow.providers.clickhouse.hooks.clickhouse import ClickHouseHook\n\n    # Define connection IDs and table names\n    postgres_conn_id = "postgres_source"  # Replace with your actual Postgres connection ID\n    clickhouse_conn_id = "clickhouse_dwh"\n    source_table = "orders"\n    target_table = "orders" # Assuming the target table has the same name\n\n    # Define source schema explicitly for clarity, though hook handles types\n    source_schema = {\n        \'order_id\': \'Int64\',\n

##### всё сохраняем в одном месте

In [161]:
with open('../templates/airflow_dag_template.py', "r", encoding="utf-8") as f:
    template = f.read()

template

'from datetime import datetime, timedelta\n\nfrom airflow.decorators import dag, task\n\n\nPROJECT_DIR = "/opt/airflow/dbt"\nDATA_PATH = f"{PROJECT_DIR}/sample"\nSEED_PATH = f"{PROJECT_DIR}/seeds"\n\nDEFAULT_ARGS = {\n    "owner": "airflow",\n    "depends_on_past": False,\n    "email_on_failure": False,\n    "retries": 1,\n    "retry_delay": timedelta(minutes=5)\n}\n\n\n@dag(\n    default_args=DEFAULT_ARGS,\n    max_active_runs=1,\n    schedule_interval="0 * * * *",\n    start_date=datetime(2025, 1, 1),\n    catchup=True\n)\ndef airflow_pipeline():\n\n    @task\n    def moving_data_from_source_to_dwh(**context) -> None:\n        pass\n\n    @task.bash\n    def build_staging_models() -> str:\n        bash_command=f"dbt run --profiles-dir {PROJECT_DIR} " \\\n                             f"--project-dir {PROJECT_DIR} " \\\n                             f"--select tag:stage" \\\n                             f"--no-version-check " \\\n        \n        return bash_command\n    \n    @task.ba

In [162]:
# ПОДСТАВЛЯЕМ schedule_interval и start_date
for line in cleaned_args_code.splitlines():
    if "schedule_interval" in line:
        template = re.sub(r'schedule_interval\s*=\s*["\']{0,1}["\']{0,1}', line, template)
    if "start_date" in line:
        template = re.sub(r'start_date\s*=\s*datetime\([^)]+\)', line, template)

In [163]:
template

'from datetime import datetime, timedelta\n\nfrom airflow.decorators import dag, task\n\n\nPROJECT_DIR = "/opt/airflow/dbt"\nDATA_PATH = f"{PROJECT_DIR}/sample"\nSEED_PATH = f"{PROJECT_DIR}/seeds"\n\nDEFAULT_ARGS = {\n    "owner": "airflow",\n    "depends_on_past": False,\n    "email_on_failure": False,\n    "retries": 1,\n    "retry_delay": timedelta(minutes=5)\n}\n\n\n@dag(\n    default_args=DEFAULT_ARGS,\n    max_active_runs=1,\n    schedule_interval="0 1 * * *"0 * * * *",\n    start_date=datetime(2025, 2, 1),\n    catchup=True\n)\ndef airflow_pipeline():\n\n    @task\n    def moving_data_from_source_to_dwh(**context) -> None:\n        pass\n\n    @task.bash\n    def build_staging_models() -> str:\n        bash_command=f"dbt run --profiles-dir {PROJECT_DIR} " \\\n                             f"--project-dir {PROJECT_DIR} " \\\n                             f"--select tag:stage" \\\n                             f"--no-version-check " \\\n        \n        return bash_command\n    \n  

In [164]:
def indent_code_block(code: str, indent: int) -> str:
    lines = code.splitlines()
    if not lines:
        return ""
    first_line = lines[0]
    indented_lines = [(" " * indent) + line if line.strip() else "" for line in lines[1:]]
    return "\n".join([first_line] + indented_lines)

cleaned_code_moving = indent_code_block(cleaned_code_moving, indent=4)
# ПОДСТАВЛЯЕМ функцию moving_data_from_source_to_dwh
template = re.sub(
    r"def moving_data_from_source_to_dwh\(\*\*context\) -> None:\n\s*pass",
    cleaned_code_moving,
    template
)

template

'from datetime import datetime, timedelta\n\nfrom airflow.decorators import dag, task\n\n\nPROJECT_DIR = "/opt/airflow/dbt"\nDATA_PATH = f"{PROJECT_DIR}/sample"\nSEED_PATH = f"{PROJECT_DIR}/seeds"\n\nDEFAULT_ARGS = {\n    "owner": "airflow",\n    "depends_on_past": False,\n    "email_on_failure": False,\n    "retries": 1,\n    "retry_delay": timedelta(minutes=5)\n}\n\n\n@dag(\n    default_args=DEFAULT_ARGS,\n    max_active_runs=1,\n    schedule_interval="0 1 * * *"0 * * * *",\n    start_date=datetime(2025, 2, 1),\n    catchup=True\n)\ndef airflow_pipeline():\n\n    @task\n    def moving_data_from_source_to_dwh(**context) -> None:\n        """\n        Извлекает данные из таблицы \'orders\' в источнике PostgreSQL\n        и загружает их в таблицу \'orders\' в ClickHouse DWH.\n\n        Предполагается, что структура таблицы \'orders\' в ClickHouse\n        соответствует схеме источника:\n        (order_id Int64, product_id Int64, timestamp DateTime, customer_id Int64, money Decimal).\n  

In [165]:
# Сохраняем итоговый файл
dag_path = "results/airflow_dag/airflow_filled.py"
with open(dag_path, "w", encoding="utf-8") as f:
    f.write(template)
print(f"DAG saved to {dag_path}")

DAG saved to results/airflow_dag/airflow_filled.py


# Генерация кода для DBT проекта

#### sources.yml

In [42]:
from typing import List, Dict, Optional
from pydantic import BaseModel

class DataSource(BaseModel):
    name: str
    description: str
    data_schema: Dict[str, str] # column_name: type
    type: str  # 'table', 'csv', 'api', etc.
    database: str
    access_method: str | None = None
    limitations: Optional[str] | None = None
    recommendations: List[str] | None = None
    connection_params: Dict[str, str] | None = None


ds_orders = DataSource(
    name="orders",
    description="Таблица заказов",
    data_schema={"order_id": "Int64",
                 "product_id": "Int64",
                 "timestamp": "datetime",
                 "customer_id": "Int64",
                 "money": "numeric"},
    type="table",
    database="PostgreSQL",
    access_method=None,
    recommendations=["использовать фильтрацию по дате (timestamp)"]
)
ds_customers = DataSource(
    name="customers",
    description="Таблица клиентов",
    data_schema={"region_id": "Int64",
                 "registration_date": "datetime",
                 "customer_id": "Int64"},
    type="table",
    database="PostgreSQL",
    access_method=None,
    )

In [13]:
columns_yaml = ""
for col_name, col_type in ds_orders.data_schema.items():
    columns_yaml += f"        - name: {col_name}\n" \
                    f"          data_type: {col_type}\n"

sources_yml = f"""version: 2

sources:
- name: exported_data
  tables:
    - name: {ds_orders.name}
      description: "{ds_orders.description}"
      columns:\n""" + f"{columns_yaml}"

In [14]:
with open("results/dbt/models/sources.yml", "w", encoding="utf-8") as f:
    f.write(sources_yml)

In [5]:
# 2 вариант
source_dict = {
    'version': 2,
    'sources': [
        {
            'name': 'exported_data',
            'tables': [
                {'name': ds_orders.name,
                 'description': ds_orders.description,
                 'columns': ds_orders.data_schema},
            ]
        }
    ]
}

In [ ]:
import yaml

with open("results/dbt/models/sources2.yml", 'w', encoding='utf-8') as f:
    yaml.dump(source_dict, f, sort_keys=False, allow_unicode=True)


#### profiles.yml


In [2]:
# 2 вариант
profiles_dict = {
    'airflow': {
      "target": "dev",
      "outputs": {
        "dev": {
          "type": "postgresql",
          "host": "localhost"
        }
      }
    }
}

import yaml

with open("results/dbt/models/profiles.yml", 'w', encoding='utf-8') as f:
    yaml.dump(profiles_dict, f, sort_keys=False, allow_unicode=True)


#### stage models

In [1]:
import re

def clean_sql_code(code_str: str) -> str:
    # убрать обрамление ``` или ```python и оставить только содержимое
    pattern = r"```(?:sql)?\n(.*?)```"
    matches = re.findall(pattern, code_str, re.DOTALL)
    if matches:
        # если несколько блоков, объединяем их через 2 перевода строки
        return "\n\n".join(match.strip() for match in matches)
    return code_str.strip()

sources = {
    'version': 2,
    'sources': [
        {
            'name': 'exported_data',
            'tables': [
                {'name': ds_orders.name + "_last_data",
                 'identifier': "orders_last_data",
                 'description': ds_orders.description},
            ]
        }
    ]
}

NameError: name 'ds_orders' is not defined

In [ ]:
sources

{'version': 2,
 'sources': [{'name': 'exported_data',
   'tables': [{'name': 'orders_last_data',
     'identifier': 'orders_last_data',
     'description': 'Таблица заказов'}]}]}

In [10]:
from langchain_core.prompts import ChatPromptTemplate
import yaml
from langchain_openai import ChatOpenAI

MODEL = "just-ai/openai-proxy/gpt-4o"
TEMPERATURE = 0

llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)

system_template = (
    "You are a data engineer. "
    "Generate a dbt model for the 'stage' layer in the medallion architecture."
    "The model should select all columns from the raw table in the analytical DWH using dbt's source() function."
    "Add a config block at the top with materialized='view'"
)

user_template = (
    "Here is the dbt sources.yml content: {sources}\n"
    "- Do not add any comments, code or explanations outside the function code.\n"
    "- Return only the SQL code."
)

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

chain = prompt_template | llm

result = chain.invoke(
    {"sources": yaml.dump(sources, allow_unicode=True)}
)

In [18]:
cleaned_sql_code = clean_sql_code(result.content)


In [19]:
stage_path = "results/dbt/models/stage.sql"
with open(stage_path, "w", encoding="utf-8") as f:
    f.write(cleaned_sql_code)

#### stage v2

In [43]:
source_dict = {
    'version': 2,
    'sources': [
        {
            'name': 'exported_data',
            'schema': 'last',
            'tables': [
                {'name': ds_orders.name,
                 'identifier': ds_orders.name + '_last_data',
                 'description': ds_orders.description,
                 'columns': ds_orders.data_schema},
                {'name': ds_customers.name,
                 'identifier': ds_customers.name + '_last_data',
                 'description': ds_customers.description,
                 'columns': ds_customers.data_schema}
            ]
        }
    ]
}

In [35]:
source_dict['sources'][0]['tables'][0]

{'name': 'orders',
 'identifier': 'orders_last_data',
 'description': 'Таблица заказов',
 'columns': {'order_id': 'Int64',
  'product_id': 'Int64',
  'timestamp': 'datetime',
  'customer_id': 'Int64',
  'money': 'numeric'}}

In [44]:
from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
import yaml
from langchain_openai import ChatOpenAI

MODEL = "just-ai/openai-proxy/gpt-4o"
TEMPERATURE = 0

llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)


class ModelMetadata(BaseModel):
    sql_code: str
    schema_yml: dict

# parser = JsonOutputParser(pydantic_object=ModelMetadata)
parser = JsonOutputParser()
    
prompt = ChatPromptTemplate.from_messages([
    ("system", """  
    Ты опытный дата-инженер, реализуешь dbt-модели для stage в трёхслойной архитектуре аналитического хранилища, построенного в Clickhouse.
    Сгенерируй dbt-модели, включая:
    1. SQL-код (вместе с config)
    2. Описание для внесения в schema.yml (включая тесты, если они необходимы)
    3. Добавь метку о времени и/или другие метаданные, полезные на слое stage
     
    Структура вывода (строго в JSON):
        {{
            "stg_model_name1": {{
                "sql_code": "SQL-код модели",
                "schema_yml": "описание модели в формате JSON для schema.yml"
            }},
            "stg_model_name2": {{
                "sql_code": "SQL-код модели",
                "schema_yml": "описание модели в формате JSON для schema.yml"
            }},
            ...
        }}
    """),
    ("user", """
    Создай dbt-модели для stage-слоя на основе sources.yml: {source}.
    Формат вывода: JSON
    """)
])

chain = prompt | llm | parser
    
result = chain.invoke({
        "source": source_dict
})

In [45]:
result

{'stg_orders': {'sql_code': "{% set source_table = source('exported_data', 'orders') %}\n\nselect\n    order_id,\n    product_id,\n    timestamp,\n    customer_id,\n    money,\n    now() as loaded_at\nfrom {{ source_table }}",
  'schema_yml': {'version': 2,
   'models': [{'name': 'stg_orders',
     'description': 'Stage layer for orders data',
     'columns': [{'name': 'order_id',
       'description': 'Unique identifier for the order',
       'tests': ['unique', 'not_null']},
      {'name': 'product_id',
       'description': 'Identifier of the product ordered',
       'tests': ['not_null']},
      {'name': 'timestamp',
       'description': 'Timestamp of the order',
       'tests': ['not_null']},
      {'name': 'customer_id',
       'description': 'Identifier of the customer who made the order',
       'tests': ['not_null']},
      {'name': 'money',
       'description': 'Amount of money involved in the order',
       'tests': ['not_null']},
      {'name': 'loaded_at',
       'descri

In [46]:
stage_path = "results/dbt/models/stage_orders.sql"
with open(stage_path, "w", encoding="utf-8") as f:
    f.write(result['stg_orders']['sql_code'])

stage_path = "results/dbt/models/stage_customers.sql"
with open(stage_path, "w", encoding="utf-8") as f:
    f.write(result['stg_customers']['sql_code'])

In [40]:
stage_schema_path = "results/dbt/models/schema_stage.yml"
with open(stage_schema_path, 'w', encoding='utf-8') as f:
    yaml.dump(result["stg_orders"]["schema_yml"], f, sort_keys=False, allow_unicode=True)
